In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
import pickle
import os
from datetime import datetime, date, timedelta

import selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [24]:
offers_db = pickle.load(open('db/offers_db', 'rb'))

In [27]:
offers_db

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
0,justjoin.it,junior,Software Developer Intern,AI Clearing,Warszawa,Not specified,6000.0,5000.0,7000.0,"[SQL, Python, Git]",https://justjoin.it/offers/ai-clearing-softwar...,2024-02-15 11:41,mazowieckie
1,justjoin.it,junior,AI/ML Engineer,MindPal,Warszawa,Praca zdalna,8500.0,7000.0,10000.0,"[AI, Machine Learning, C++]",https://justjoin.it/offers/mindpal-ai-ml-engin...,2024-02-15 11:41,mazowieckie
2,justjoin.it,junior,Junior Data Management Specialist,DXC Technology,Warszawa,Praca zdalna,NaN,NaN,NaN,"[data analytics, SQL, Python]",https://justjoin.it/offers/dxc-technology-juni...,2024-02-15 11:41,mazowieckie
3,justjoin.it,junior,Junior Python / DevOps Engineer,digatus,Warszawa,Praca zdalna,11000.0,9000.0,13000.0,"[Docker, DevOps, Python]",https://justjoin.it/offers/digatus-junior-pyth...,2024-02-15 11:41,mazowieckie
4,justjoin.it,junior,"Software Engineer, Early Career",Affirm,Warszawa,Not specified,16450.0,16450.0,16450.0,"[JavaScript, Kotlin, C++]",https://justjoin.it/offers/affirm-software-eng...,2024-02-15 11:41,mazowieckie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,pracuj.pl,mid,Specjalista ds. Narzędzi i Analiz BI,PKO Bank Polski SA,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[Python, SQL]",https://www.pracuj.pl/praca/specjalista-ds-nar...,2024-02-28 10:03,mazowieckie
1642,pracuj.pl,junior,Junior Data Scientist,A.T. Kearney Sp. z o.o.,Warszawa,Praca hybrydowa,10750.0,9500.0,12000.0,"[SQL, Python]",https://www.pracuj.pl/praca/junior-data-scient...,2024-02-28 10:03,mazowieckie
1643,pracuj.pl,junior,Młodszy Analityk Danych,Hart Sp. z o.o.,Wrocław,Praca zdalna,NaN,NaN,NaN,"[SQL, Oracle]",https://www.pracuj.pl/praca/mlodszy-analityk-d...,2024-02-28 10:03,dolnośląskie
1644,pracuj.pl,mid,Analityk IT / Feature Engineer,DomData AG Sp. z o.o.,Katowice,Praca zdalna,NaN,NaN,NaN,"[UML, BPMN, SQL, SOAP]",https://www.pracuj.pl/praca/analityk-it-featur...,2024-02-28 10:03,śląskie


In [30]:
current_dir = os.getcwd()
driver_path = os.path.join(current_dir, 'app/chromedriver.exe')
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)

In [31]:
url='https://it.pracuj.pl/praca?itth=37'
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [50]:
i=0
for offer in soup.find_all('div', class_="be8lukl core_po9665q"):
    whole_offer = offer.find('div', class_="c1fljezf")
    offer_details = whole_offer.find('div', class_="c1wygkax")
    location = offer_details.h5.text
    print(f'{i}: {location}')
    i+=1

0: Warszawa, Mokotów
1: Wrocław, Krzyki
2: Wrocław, Krzyki
3: Kraków, Dębniki
4: Wrocław, Krzyki
5: Kraków
6: Warszawa, Wola
7: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Mokotów
8: Kraków, Grzegórzki
9: Kraków, Grzegórzki
10: Kraków, Prądnik Czerwony
11: Warszawa, Praga-Południe
12: Kraków, Dębniki
13: Warszawa
14: Warszawa, Żoliborz
15: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Praga-Północ
16: Warszawa, Ochota
17: Warszawa
18: Katowice
19: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Wola
20: Warszawa, Mokotów
21: 5 lokalizacji
22: 5 lokalizacji
23: 5 lokalizacji
24: 5 lokalizacji
25: 3 lokalizacje
26: 4 lokalizacje
27: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Warszawa, Wola
28: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:Rybnik, Smolna
29: Wrocław, Stare Miasto
30: Warszawa
31: Miejsce pracy:Cała Polska (praca zdalna)Siedziba firmy:5 lokalizacji
32: Miejsce pracy:Cała Polska (praca zdalna)Sied

In [51]:
offer = soup.find_all('div', class_="be8lukl core_po9665q")[21]
whole_offer = offer.find('div', class_="c1fljezf")
offer_details = whole_offer.find('div', class_="c1wygkax")
keywords = whole_offer.find_all('div', class_='b1fdzgc4')
link = offer_details.a['href']

In [52]:
location = offer_details.h5.text

In [53]:
offer_details.h5.text

'5 lokalizacji'

In [54]:
link

'https://pracodawcy.pracuj.pl/company/1074142237'